# Scraper de Trustpilot (Categoría: Viajes y Vacaciones)

Este notebook extrae información de empresas y reseñas de https://es.trustpilot.com/categories/travel_vacation usando Selenium.

## Variables que extrae:
- ID de la reseña, Dominio, Nombre de la empresa, Categorías, Calificación de la empresa
- Fecha de la reseña, Nombre del cliente, Puntuación del cliente, Texto de la reseña
- Columnas vacías para LLM: Idioma, Sentimiento, Emoción, Género del cliente, Tema principal, Palabras clave, Tipo de cliente, Tipo de turista, Tipo de grupo, ¿Analizado?


In [1]:
# Importar librerías necesarias
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re
from datetime import datetime
from tqdm import tqdm
import json
import hashlib


In [2]:
# Configuración del driver de Chrome
def setup_driver(headless=False):
    """Configura y retorna el driver de Chrome con las opciones necesarias"""
    chrome_options = Options()
    
    # Modo headless opcional
    if headless:
        chrome_options.add_argument('--headless')
    
    # Opciones para evitar detección
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    
    # Desactivar imágenes para cargar más rápido (opcional)
    # prefs = {"profile.managed_default_content_settings.images": 2}
    # chrome_options.add_experimental_option("prefs", prefs)
    
    # User agent realista
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36')
    
    # Tamaño de ventana
    chrome_options.add_argument('--window-size=1920,1080')
    
    # Otras opciones para parecer más humano
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--proxy-server="direct://"')
    chrome_options.add_argument('--proxy-bypass-list=*')
    chrome_options.add_argument('--start-maximized')
    
    try:
        # Inicializar driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        # Ejecutar JavaScript para ocultar webdriver
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        
        driver.implicitly_wait(10)
        print("✅ Driver Chrome iniciado correctamente")
        return driver
        
    except Exception as e:
        print(f"❌ Error al iniciar Chrome: {e}")
        raise e


In [3]:
# Función para delays aleatorios más humanos
import random

def random_delay(min_seconds=1, max_seconds=3):
    """Pausa aleatoria para parecer más humano"""
    delay = random.uniform(min_seconds, max_seconds)
    time.sleep(delay)


In [4]:
# Funciones auxiliares
def generate_review_id(company_name, review_date, customer_name, review_text):
    """Genera un ID único para cada reseña"""
    content = f"{company_name}{review_date}{customer_name}{review_text[:50]}"
    return hashlib.md5(content.encode()).hexdigest()[:12]

def parse_date(date_str):
    """Convierte la fecha del formato de Trustpilot a formato estándar"""
    try:
        # Mapeo de meses en español
        meses = {
            'enero': 'January', 'febrero': 'February', 'marzo': 'March',
            'abril': 'April', 'mayo': 'May', 'junio': 'June',
            'julio': 'July', 'agosto': 'August', 'septiembre': 'September',
            'octubre': 'October', 'noviembre': 'November', 'diciembre': 'December'
        }
        
        # Reemplazar mes español por inglés
        for mes_esp, mes_eng in meses.items():
            date_str = date_str.replace(mes_esp, mes_eng)
        
        # Parsear fecha
        return pd.to_datetime(date_str, format='%d de %B de %Y')
    except:
        return None

def scroll_to_load_reviews(driver, max_scrolls=10):
    """Hace scroll para cargar más reseñas"""
    last_height = driver.execute_script("return document.body.scrollHeight")
    scrolls = 0
    
    while scrolls < max_scrolls:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
            
        last_height = new_height
        scrolls += 1


In [5]:
# Función de debugging para analizar la estructura de la página
def debug_page_structure(driver, url):
    """Analiza la estructura de la página para encontrar selectores correctos"""
    print(f"\n🔍 DEBUGGING: Analizando estructura de {url}")
    driver.get(url)
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Buscar todos los enlaces que podrían ser de empresas
    review_links = soup.find_all('a', href=re.compile('/review/'))
    print(f"📌 Enlaces con /review/: {len(review_links)}")
    if review_links:
        print("   Primeros 3 ejemplos:")
        for i, link in enumerate(review_links[:3]):
            print(f"   - {link.get('href')} | Texto: {link.text.strip()[:50]}")
    
    # Buscar elementos article
    articles = soup.find_all('article')
    print(f"📌 Elementos <article>: {len(articles)}")
    
    # Buscar elementos con clases que contengan 'business' o 'company'
    business_elements = soup.find_all(class_=re.compile('business|company', re.I))
    print(f"📌 Elementos con clases 'business/company': {len(business_elements)}")
    
    # Buscar elementos con data attributes
    data_attrs = soup.find_all(attrs={'data-business-unit-card': True})
    print(f"📌 Elementos con data-business-unit-card: {len(data_attrs)}")
    
    # Guardar HTML para análisis manual
    with open('trustpilot_debug.html', 'w', encoding='utf-8') as f:
        f.write(str(soup.prettify()))
    print("💾 HTML guardado en trustpilot_debug.html")
    
    # Tomar screenshot
    driver.save_screenshot('trustpilot_debug.png')
    print("📸 Screenshot guardado en trustpilot_debug.png")
    
    return soup


In [6]:
# Función para extraer información de las empresas
def get_companies_from_category(driver, category_url, max_pages=75):
    """Extrae información de empresas de una categoría"""
    companies = []
    
    for page in range(1, max_pages + 1):
        url = f"{category_url}?page={page}"
        print(f"🔍 Accediendo a: {url}")
        driver.get(url)
        time.sleep(5)  # Aumentar tiempo de espera
        
        # Guardar HTML para debugging
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Diferentes selectores posibles para las tarjetas de empresas
        # ORDEN MODIFICADO: El selector 'paper_paper__' causaba duplicados (encontraba los mismos 10 elementos en cada página)
        selectors = [
            {'tag': 'div', 'attrs': {'data-business-unit-card': True}},
            {'tag': 'article', 'attrs': {'class': re.compile('styles_businessUnitCard')}},
            {'tag': 'div', 'attrs': {'class': re.compile('styles_businessUnitCard')}},
            # Usar directamente los enlaces que SÍ funcionan (debug muestra ~30 por página)
            {'tag': 'a', 'attrs': {'href': re.compile('/review/[^/?]+$')}},  # Solo enlaces que terminan con dominio
            # ELIMINADO: {'tag': 'div', 'attrs': {'class': re.compile('paper_paper__')}}, # ¡Este causaba los duplicados!
        ]
        
        company_cards = []
        for selector in selectors:
            company_cards = soup.find_all(selector['tag'], attrs=selector['attrs'])
            if company_cards:
                print(f"✅ Encontradas {len(company_cards)} tarjetas usando selector: {selector}")
                break
        
        if not company_cards:
            print(f"⚠️ No se encontraron tarjetas de empresas en la página {page}")
            print("🔍 Guardando screenshot para debugging...")
            driver.save_screenshot(f"trustpilot_page_{page}_debug.png")
            
            # Buscar enlaces alternativos
            all_links = soup.find_all('a', href=re.compile('/review/'))
            if all_links:
                print(f"📌 Encontrados {len(all_links)} enlaces de review")
            
            # Si es la primera página, intentar con wait más específico
            if page == 1:
                try:
                    print("⏳ Esperando carga completa de la página...")
                    WebDriverWait(driver, 15).until(
                        EC.presence_of_element_located((By.TAG_NAME, "article"))
                    )
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    company_cards = soup.find_all('article')
                    if company_cards:
                        print(f"✅ Encontrados {len(company_cards)} articles después de esperar")
                except:
                    pass
            
            if not company_cards:
                continue
        
        # Si son enlaces directos, procesarlos de manera diferente
        if company_cards and company_cards[0].name == 'a':
            for link in company_cards:
                try:
                    company_url = link.get('href', '')
                    if '/review/' not in company_url:
                        continue
                        
                    if not company_url.startswith('http'):
                        company_url = f"https://es.trustpilot.com{company_url}"
                    
                    # Obtener dominio primero para limpiar el nombre
                    domain = company_url.split('/')[-1].split('?')[0]  # Remover parámetros
                    
                    # Obtener el nombre de la empresa con mejores estrategias
                    company_name = ""
                    
                    # Estrategia 1: Buscar en el texto del enlace y limpiar
                    link_text = link.get_text(separator=' ', strip=True)
                    if link_text:
                        # Limpiar texto común no deseado
                        company_name = link_text.replace('Más relevantes', '').replace('Más relevante', '').strip()
                        
                        # Si el dominio está en el texto, usar lo que está antes
                        if domain in company_name:
                            parts = company_name.split(domain)
                            if parts and parts[0]:
                                company_name = parts[0].strip()
                        
                        # Si hay números (rating), cortar antes del primer número
                        match = re.search(r'^([^0-9]+?)(?:\d|$)', company_name)
                        if match:
                            company_name = match.group(1).strip()
                    
                    # Estrategia 2: Buscar en elementos hijos si no encontramos nombre
                    if not company_name or len(company_name) < 2:
                        name_elem = link.find(['span', 'p', 'h2', 'h3'])
                        if name_elem:
                            company_name = name_elem.text.strip()
                    
                    # Fallback: usar el dominio limpio
                    if not company_name or len(company_name) < 2:
                        company_name = domain.replace('.com', '').replace('.es', '').replace('-', ' ').title()
                    
                    # Buscar información adicional en el contenedor padre
                    parent = link.parent
                    rating = "N/A"
                    num_reviews = "0"
                    
                    if parent:
                        # Buscar calificación
                        rating_elem = parent.find(text=re.compile(r'\d+[,\.]\d+'))
                        if rating_elem:
                            rating = re.search(r'\d+[,\.]\d+', rating_elem).group()
                        
                        # Buscar número de reseñas
                        reviews_elem = parent.find(text=re.compile(r'\d+\s*(reseñas?|reviews?|opiniones?)'))
                        if reviews_elem:
                            num_match = re.search(r'(\d+)', reviews_elem)
                            if num_match:
                                num_reviews = num_match.group(1)
                    
                    companies.append({
                        'company_name': company_name,
                        'domain': domain,
                        'company_url': company_url,
                        'rating': rating,
                        'num_reviews': num_reviews,
                        'categories': 'travel_vacation'
                    })
                    
                except Exception as e:
                    print(f"Error procesando enlace: {e}")
                    continue
        else:
            # Procesar tarjetas normales
            for card in company_cards:
                try:
                    # Buscar enlace principal
                    link_elem = card.find('a', href=re.compile('/review/'))
                    if not link_elem:
                        continue
                    
                    company_url = link_elem.get('href', '')
                    if not company_url.startswith('http'):
                        company_url = f"https://es.trustpilot.com{company_url}"
                    
                    # Nombre de la empresa - múltiples estrategias
                    company_name = None
                    name_selectors = [
                        ('p', {'class': re.compile('typography_heading')}),
                        ('span', {'class': re.compile('typography_heading')}),
                        ('h2', {}),
                        ('h3', {}),
                        ('p', {'class': re.compile('displayName')}),
                        ('a', {}),  # El mismo enlace
                    ]
                    
                    for tag, attrs in name_selectors:
                        name_elem = card.find(tag, attrs)
                        if name_elem and name_elem.text.strip():
                            company_name = name_elem.text.strip()
                            break
                    
                    if not company_name:
                        company_name = link_elem.text.strip() or company_url.split('/')[-1]
                    
                    # Dominio
                    domain = company_url.split('/')[-1] if company_url else "N/A"
                    
                    # Calificación
                    rating = "N/A"
                    rating_patterns = [r'\d+[,\.]\d+', r'\d+\.\d+', r'\d+,\d+']
                    for pattern in rating_patterns:
                        rating_elem = card.find(text=re.compile(pattern))
                        if rating_elem:
                            match = re.search(pattern, rating_elem)
                            if match:
                                rating = match.group()
                                break
                    
                    # Número de reseñas
                    num_reviews = "0"
                    review_patterns = [
                        r'(\d+)\s*(reseñas?|opiniones?|reviews?)',
                        r'(\d+)\s*total',
                        r'\((\d+)\)'
                    ]
                    
                    for pattern in review_patterns:
                        reviews_elem = card.find(text=re.compile(pattern, re.IGNORECASE))
                        if reviews_elem:
                            match = re.search(r'\d+', reviews_elem)
                            if match:
                                num_reviews = match.group()
                                break
                    
                    if company_name and company_name != "N/A":
                        companies.append({
                            'company_name': company_name,
                            'domain': domain,
                            'company_url': company_url,
                            'rating': rating,
                            'num_reviews': num_reviews,
                            'categories': 'travel_vacation'
                        })
                        
                except Exception as e:
                    print(f"Error al procesar tarjeta: {e}")
                    continue
        
        print(f"📊 Página {page}: {len(companies)} empresas acumuladas")
    
    # Eliminar duplicados
    seen = set()
    unique_companies = []
    for company in companies:
        if company['company_url'] not in seen:
            seen.add(company['company_url'])
            unique_companies.append(company)
    
    print(f"\n✅ Total de empresas únicas encontradas: {len(unique_companies)}")
    if unique_companies:
        print("📋 Primeras 3 empresas:")
        for i, company in enumerate(unique_companies[:3]):
            print(f"   {i+1}. {company['company_name']} - {company['rating']} ({company['num_reviews']} reseñas)")
    
    return unique_companies


In [7]:
# TEST: Ejecutar debugging para ver qué está pasando
driver = setup_driver(headless=False)  # Usar headless=True si no quieres ver el navegador
try:
    # Analizar la página de categoría
    category_url = "https://es.trustpilot.com/categories/travel_vacation"
    debug_page_structure(driver, category_url)
    
    print("\n" + "="*50)
    print("🔍 Intentando acceso directo a una empresa conocida...")
    
    # Intentar acceder directamente a una empresa conocida
    test_company_url = "https://es.trustpilot.com/review/www.booking.com"
    driver.get(test_company_url)
    time.sleep(3)
    
    if "booking" in driver.current_url.lower():
        print("✅ Acceso exitoso a página de empresa")
    else:
        print("❌ Redirigido o bloqueado")
        print(f"URL actual: {driver.current_url}")
        
finally:
    driver.quit()
    print("\n🔚 Test completado")


✅ Driver Chrome iniciado correctamente

🔍 DEBUGGING: Analizando estructura de https://es.trustpilot.com/categories/travel_vacation
📌 Enlaces con /review/: 30
   Primeros 3 ejemplos:
   - /review/guruwalk.com | Texto: Más relevantesGuruwalk - Free toursguruwalk.com4,9
   - /review/dayuse.es | Texto: Más relevantesDayuse.esdayuse.es4,9862 opiniones5 
   - /review/bookaris.com | Texto: Más relevantesBookaris.combookaris.com4,9514 opini
📌 Elementos <article>: 0
📌 Elementos con clases 'business/company': 190
📌 Elementos con data-business-unit-card: 0
💾 HTML guardado en trustpilot_debug.html
📸 Screenshot guardado en trustpilot_debug.png

🔍 Intentando acceso directo a una empresa conocida...
✅ Acceso exitoso a página de empresa

🔚 Test completado


In [8]:
# Función para extraer reseñas de una empresa con paginación
def get_reviews_from_company(driver, company_info, max_review_pages=3):
    """Extrae reseñas de una empresa específica con paginación"""
    reviews = []
    
    for page in range(1, max_review_pages + 1):
        # Construir URL con paginación
        if page == 1:
            review_url = company_info['company_url']
        else:
            review_url = f"{company_info['company_url']}?page={page}"
        
        print(f"   📄 Página {page} de reseñas: {review_url}")
        driver.get(review_url)
        random_delay(2, 4)  # Delay aleatorio más humano
        
        # Hacer scroll para cargar más reseñas en la página actual
        scroll_to_load_reviews(driver, max_scrolls=3)
        
        # Obtener HTML actualizado
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Buscar reseñas
        review_cards = soup.find_all('article', class_=re.compile('paper_paper__'))
        
        if not review_cards:
            print(f"   ⚠️ No se encontraron reseñas en la página {page}")
            break
        
        page_reviews = 0
        for card in review_cards:
            try:
                # Nombre del cliente
                customer_elem = card.find('span', attrs={'data-consumer-name-typography': 'true'})
                customer_name = customer_elem.text.strip() if customer_elem else "Anónimo"
                
                # Fecha de la reseña
                date_elem = card.find('time')
                review_date = date_elem.get('datetime', '') if date_elem else ""
                if not review_date:
                    date_text = date_elem.text.strip() if date_elem else ""
                    review_date = parse_date(date_text)
                
                # Puntuación (estrellas)
                rating_elem = card.find('div', attrs={'data-service-review-rating': True})
                if rating_elem:
                    rating_attr = rating_elem.get('data-service-review-rating', '0')
                    customer_score = int(rating_attr) if rating_attr.isdigit() else 0
                else:
                    # Buscar alternativa
                    star_elem = card.find('img', alt=re.compile('Valorado con'))
                    if star_elem:
                        alt_text = star_elem.get('alt', '')
                        score_match = re.search(r'(\d+)', alt_text)
                        customer_score = int(score_match.group(1)) if score_match else 0
                    else:
                        customer_score = 0
                
                # Texto de la reseña
                review_elem = card.find('p', attrs={'data-service-review-text-typography': 'true'})
                review_text = review_elem.text.strip() if review_elem else ""
                
                # Generar ID único
                review_id = generate_review_id(
                    company_info['company_name'], 
                    str(review_date), 
                    customer_name, 
                    review_text
                )
                
                reviews.append({
                    'review_id': review_id,
                    'domain': company_info['domain'],
                    'company_name': company_info['company_name'],
                    'categories': company_info['categories'],
                    'company_rating': company_info['rating'],
                    'review_date': review_date,
                    'customer_name': customer_name,
                    'customer_score': customer_score,
                    'review_text': review_text,
                    # Columnas vacías para análisis LLM
                    'language': '',
                    'sentiment': '',
                    'emotion': '',
                    'customer_gender': '',
                    'main_topic': '',
                    'keywords': '',
                    'customer_type': '',
                    'tourist_type': '',
                    'group_type': '',
                    'analyzed': False
                })
                page_reviews += 1
                
            except Exception as e:
                print(f"   ❌ Error al procesar reseña: {e}")
                continue
        
        print(f"   ✅ Página {page}: {page_reviews} reseñas extraídas")
        
        # Si no hay más páginas, salir del loop
        if page_reviews == 0:
            break
            
        # Verificar si hay botón "Siguiente" para continuar
        try:
            next_button = soup.find('a', {'aria-label': re.compile('Next|Siguiente', re.I)})
            if not next_button or 'disabled' in str(next_button.get('class', [])):
                print(f"   🏁 No hay más páginas de reseñas")
                break
        except:
            pass
    
    print(f"📊 Total: {len(reviews)} reseñas extraídas de {company_info['company_name']}")
    return reviews


In [9]:
# Función principal del scraper
def scrape_trustpilot_travel(max_companies=10, max_review_pages_per_company=3, max_company_pages=3):
    """
    Función principal que ejecuta todo el proceso de scraping
    
    Parámetros:
    - max_companies: Número máximo de empresas a procesar
    - max_review_pages_per_company: Número máximo de páginas de reseñas por empresa
    - max_company_pages: Número máximo de páginas de la categoría a recorrer
    """
    # URL de la categoría de viajes y vacaciones
    category_url = "https://es.trustpilot.com/categories/travel_vacation"
    
    # Inicializar driver
    print("🚀 Iniciando navegador...")
    driver = setup_driver()
    
    try:
        # Obtener lista de empresas
        print(f"\n🔍 Buscando empresas en la categoría de viajes...")
        print(f"   • Páginas de categoría a recorrer: {max_company_pages}")
        companies = get_companies_from_category(driver, category_url, max_pages=max_company_pages)
        
        # Limitar número de empresas
        companies = companies[:max_companies]
        print(f"\n📋 Total de empresas a procesar: {len(companies)}")
        print(f"   • Páginas de reseñas por empresa: {max_review_pages_per_company}")
        
        # Extraer reseñas de cada empresa
        all_reviews = []
        
        for i, company in enumerate(tqdm(companies, desc="Procesando empresas")):
            print(f"\n[{i+1}/{len(companies)}] 🏢 Procesando: {company['company_name']}")
            
            try:
                reviews = get_reviews_from_company(driver, company, max_review_pages=max_review_pages_per_company)
                all_reviews.extend(reviews)
                
                # Pausa entre empresas para evitar bloqueos
                random_delay(2, 4)
                
            except Exception as e:
                print(f"❌ Error al procesar {company['company_name']}: {e}")
                continue
        
        # Crear DataFrame
        df_reviews = pd.DataFrame(all_reviews)
        
        # Guardar resultados
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"trustpilot_travel_reviews_{timestamp}.csv"
        df_reviews.to_csv(filename, index=False, encoding='utf-8-sig')
        
        print(f"\n✅ Scraping completado!")
        print(f"📊 Total de reseñas extraídas: {len(all_reviews)}")
        print(f"💾 Archivo guardado: {filename}")
        
        # Mostrar estadísticas
        if len(df_reviews) > 0:
            print(f"\n📈 Estadísticas:")
            print(f"   • Empresas únicas: {df_reviews['company_name'].nunique()}")
            print(f"   • Promedio de reseñas por empresa: {len(df_reviews) / df_reviews['company_name'].nunique():.1f}")
            print(f"   • Distribución de puntuaciones:")
            score_dist = df_reviews['customer_score'].value_counts().sort_index()
            for score, count in score_dist.items():
                print(f"     ⭐ {score}: {count} reseñas ({count/len(df_reviews)*100:.1f}%)")
        
        return df_reviews
        
    finally:
        # Cerrar navegador
        driver.quit()
        print("\n🔚 Navegador cerrado.")


In [10]:
# Ejecutar el scraper con configuración básica
# NOTA: Ajusta los parámetros según tus necesidades
df_results = scrape_trustpilot_travel(
    max_companies=70,                    # Número de empresas a procesar
    max_review_pages_per_company=1,     # Páginas de reseñas por empresa (cada página tiene ~20 reseñas)
    max_company_pages=5                 # Páginas de la categoría a recorrer (cada página tiene ~10 empresas)
)


🚀 Iniciando navegador...
✅ Driver Chrome iniciado correctamente

🔍 Buscando empresas en la categoría de viajes...
   • Páginas de categoría a recorrer: 5
🔍 Accediendo a: https://es.trustpilot.com/categories/travel_vacation?page=1
✅ Encontradas 29 tarjetas usando selector: {'tag': 'a', 'attrs': {'href': re.compile('/review/[^/?]+$')}}
📊 Página 1: 29 empresas acumuladas
🔍 Accediendo a: https://es.trustpilot.com/categories/travel_vacation?page=2


/var/folders/2h/j25t7r2x2gz01h_zbbny8klh0000gn/T/ipykernel_9515/4051170837.py:112: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  rating_elem = parent.find(text=re.compile(r'\d+[,\.]\d+'))
/var/folders/2h/j25t7r2x2gz01h_zbbny8klh0000gn/T/ipykernel_9515/4051170837.py:117: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  reviews_elem = parent.find(text=re.compile(r'\d+\s*(reseñas?|reviews?|opiniones?)'))


✅ Encontradas 30 tarjetas usando selector: {'tag': 'a', 'attrs': {'href': re.compile('/review/[^/?]+$')}}
📊 Página 2: 59 empresas acumuladas
🔍 Accediendo a: https://es.trustpilot.com/categories/travel_vacation?page=3
✅ Encontradas 30 tarjetas usando selector: {'tag': 'a', 'attrs': {'href': re.compile('/review/[^/?]+$')}}
📊 Página 3: 89 empresas acumuladas
🔍 Accediendo a: https://es.trustpilot.com/categories/travel_vacation?page=4
✅ Encontradas 30 tarjetas usando selector: {'tag': 'a', 'attrs': {'href': re.compile('/review/[^/?]+$')}}
📊 Página 4: 119 empresas acumuladas
🔍 Accediendo a: https://es.trustpilot.com/categories/travel_vacation?page=5
✅ Encontradas 30 tarjetas usando selector: {'tag': 'a', 'attrs': {'href': re.compile('/review/[^/?]+$')}}
📊 Página 5: 149 empresas acumuladas

✅ Total de empresas únicas encontradas: 102
📋 Primeras 3 empresas:
   1. Guruwalk - Free tours - 4,9 (0 reseñas)
   2. Dayuse.es - 4,9 (0 reseñas)
   3. Bookaris.com - 4,9 (0 reseñas)

📋 Total de empresas 

Procesando empresas:   0%|          | 0/70 [00:00<?, ?it/s]


[1/70] 🏢 Procesando: Guruwalk - Free tours
   📄 Página 1 de reseñas: https://es.trustpilot.com/review/guruwalk.com
   ✅ Página 1: 20 reseñas extraídas
📊 Total: 20 reseñas extraídas de Guruwalk - Free tours


...


[69/70] 🏢 Procesando: Pilots Center
   📄 Página 1 de reseñas: https://es.trustpilot.com/review/pilotscenter.com
   ✅ Página 1: 20 reseñas extraídas
📊 Total: 20 reseñas extraídas de Pilots Center


Procesando empresas:  99%|█████████▊| 69/70 [14:47<00:10, 10.01s/it]


[70/70] 🏢 Procesando: Arenatours
   📄 Página 1 de reseñas: https://es.trustpilot.com/review/arenatours.com
   ✅ Página 1: 20 reseñas extraídas
📊 Total: 20 reseñas extraídas de Arenatours


Procesando empresas: 100%|██████████| 70/70 [14:57<00:00, 12.82s/it]



✅ Scraping completado!
📊 Total de reseñas extraídas: 1359
💾 Archivo guardado: trustpilot_travel_reviews_20250701_181108.csv

📈 Estadísticas:
   • Empresas únicas: 70
   • Promedio de reseñas por empresa: 19.4
   • Distribución de puntuaciones:
     ⭐ 1: 158 reseñas (11.6%)
     ⭐ 2: 33 reseñas (2.4%)
     ⭐ 3: 40 reseñas (2.9%)
     ⭐ 4: 133 reseñas (9.8%)
     ⭐ 5: 995 reseñas (73.2%)

🔚 Navegador cerrado.


In [12]:
# Ver estadísticas del dataset
if 'df_results' in locals():
    print("📊 Resumen del dataset:")
    print(f"Total de reseñas: {len(df_results)}")
    print(f"Empresas únicas: {df_results['company_name'].nunique()}")
    print(f"\nDistribución de puntuaciones:")
    print(df_results['customer_score'].value_counts().sort_index())
    
    # Mostrar primeras filas
    print("\n📋 Primeras 5 reseñas:")
    display(df_results.head())


📊 Resumen del dataset:
Total de reseñas: 1359
Empresas únicas: 70

Distribución de puntuaciones:
customer_score
1    158
2     33
3     40
4    133
5    995
Name: count, dtype: int64

📋 Primeras 5 reseñas:


,review_id,domain,company_name,categories,company_rating,review_date,customer_name,customer_score,review_text,language,sentiment,emotion,customer_gender,main_topic,keywords,customer_type,tourist_type,group_type,analyzed
0,1a7e7f41b5d6,guruwalk.com,Guruwalk - Free tours,travel_vacation,"4,9",2025-07-01T23:39:24.000Z,Italo Constenla Carrillo,5,"Marcelo un gran guía , una muy grata experienc...",,,,,,,,,,False
1,f17c6e5f270f,guruwalk.com,Guruwalk - Free tours,travel_vacation,"4,9",2025-07-01T23:21:03.000Z,Noelia Tannous,5,"Laura Trovato ha sido una magnífica guía, muy ...",,,,,,,,,,False
2,218f31094ed0,guruwalk.com,Guruwalk - Free tours,travel_vacation,"4,9",2025-07-01T20:37:20.000Z,Cliente,5,Muy recomendable,,,,,,,,,,False
3,105e9dfd9448,guruwalk.com,Guruwalk - Free tours,travel_vacation,"4,9",2025-07-01T19:35:47.000Z,Manuel González,5,Muy buena plataforma para hacer tours por dife...,,,,,,,,,,False
4,a77cd0af2ebc,guruwalk.com,Guruwalk - Free tours,travel_vacation,"4,9",2025-07-01T09:36:47.000Z,RS,5,Tour entretenido y ameno por el casco antiguo ...,,,,,,,,,,False


In [13]:
# Función para cargar y combinar múltiples archivos CSV
def combine_csv_files(pattern="trustpilot_travel_reviews_*.csv"):
    """Combina múltiples archivos CSV en un solo DataFrame"""
    import glob
    
    files = glob.glob(pattern)
    if not files:
        print("No se encontraron archivos CSV")
        return None
    
    dfs = []
    for file in files:
        df = pd.read_csv(file, encoding='utf-8-sig')
        dfs.append(df)
    
    combined_df = pd.concat(dfs, ignore_index=True)
    
    # Eliminar duplicados basados en review_id
    combined_df = combined_df.drop_duplicates(subset=['review_id'])
    
    print(f"Archivos combinados: {len(files)}")
    print(f"Total de reseñas únicas: {len(combined_df)}")
    
    return combined_df


In [ ]:
'''
# Función para filtrar reseñas por puntuación o fecha
def filter_reviews(df, min_score=None, max_score=None, start_date=None, end_date=None):
    """Filtra reseñas según criterios específicos"""
    filtered_df = df.copy()
    
    if min_score is not None:
        filtered_df = filtered_df[filtered_df['customer_score'] >= min_score]
    
    if max_score is not None:
        filtered_df = filtered_df[filtered_df['customer_score'] <= max_score]
    
    if start_date is not None:
        filtered_df['review_date'] = pd.to_datetime(filtered_df['review_date'])
        filtered_df = filtered_df[filtered_df['review_date'] >= start_date]
    
    if end_date is not None:
        filtered_df['review_date'] = pd.to_datetime(filtered_df['review_date'])
        filtered_df = filtered_df[filtered_df['review_date'] <= end_date]
    
    print(f"Reseñas filtradas: {len(filtered_df)} de {len(df)}")
    return filtered_df

# Ejemplo de uso:
# df_filtered = filter_reviews(df_results, min_score=4, start_date='2024-01-01')
'''